Ripped from https://github.com/mees/calvin/blob/main/RL_with_CALVIN.ipynb

In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('/home/grail/willaria_research/hobbes/calvin_env'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['/home/grail/willaria_research/hobbes', '/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages/git/ext/gitdb', '', '/home/grail/interbotix_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/home/grail/miniconda3/envs/calvin_env/lib/python38.zip', '/home/grail/miniconda3/envs/calvin_env/lib/python3.8', '/home/grail/miniconda3/envs/calvin_env/lib/python3.8/lib-dynload', '/home/grail/.local/lib/python3.8/site-packages', '/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages', '/home/grail/willaria_research/hobbes/calvin_env/tacto', '/home/grail/willaria_research/hobbes/calvin_models', '/home/grail/willaria_research/hobbes/calvin_env']


In [20]:
import time
import hydra
import numpy as np
import cv2
from hydra import initialize, compose
from calvin_env.envs.play_table_env import PlayTableSimEnv

In [21]:

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  print(cfg.env)

{'_target_': 'calvin_env.envs.play_table_env.PlayTableSimEnv', '_recursive_': False, 'cameras': '${cameras}', 'seed': 0, 'bullet_time_step': 240.0, 'use_vr': False, 'show_gui': False, 'robot_cfg': '${robot}', 'scene_cfg': '${scene}', 'use_scene_info': True, 'use_egl': False, 'control_freq': 30}


This cell below runs code to show images from some static viewpoint over a few random steps. If it says "Environment View is ready", just click on that.

In [22]:
env = hydra.utils.instantiate(cfg.env)
observation = env.reset()
#The observation is given as a dictionary with different values
print(observation.keys())
for i in range(5):
  # The action consists in a pose displacement (position and orientation)
  action_displacement = np.random.uniform(low=-10, high=10, size=6)
  # And a binary gripper action, -1 for closing and 1 for oppening
  action_gripper = np.random.choice([-1, 1], size=1)
  action = np.concatenate((action_displacement, action_gripper), axis=-1)
  print(action)
  observation, reward, done, info = env.step(action)
  rgb = env.render(mode="rgb_array")[:,:,::-1]
  cv2.imshow("Environment View", rgb)
  cv2.waitKey(0)
cv2.destroyAllWindows()

WARNING uncommitted modified files: calvin_env/scene/play_table_scene.py
dict_keys(['rgb_obs', 'depth_obs', 'robot_obs', 'scene_obs'])
[ 6.14254175 -5.66131452 -1.2899336  -1.3351189   8.95277493  5.88986544
  1.        ]
[ 8.45084953  9.10294874  2.62781913  2.66253597  9.27097397 -3.3569961
 -1.        ]
[-7.51732288  5.83584518  2.26402442  8.85227198  2.03328494  7.89791061
 -1.        ]
[ 4.06837335 -4.61540209  5.45527054  8.78953424 -5.99458083 -5.76692158
 -1.        ]
[-1.92352992 -1.2348122   9.08938106  2.65677776  9.02139711  9.55715894
  1.        ]


In [23]:
%cd /home/grail/willaria_research/hobbes/dataset/calvin_debug_dataset/training

/home/grail/willaria_research/hobbes/dataset/calvin_debug_dataset/training


In [24]:
lang = np.load("lang_annotations/auto_lang_ann.npy", allow_pickle=True)
lang = dict(enumerate(lang.flatten(), 1))[1]
lang

{'language': {'ann': ['move the light switch to turn on the yellow light',
   'sweep the pink block to the right',
   'place the block in the sliding cabinet',
   'pick up the red block from the table',
   'in the slider grasp the blue block',
   'slide down the switch',
   'pick up the red block on the table',
   'place in slider',
   'turn off the light bulb'],
  'task': ['turn_on_lightbulb',
   'push_pink_block_right',
   'place_in_slider',
   'lift_red_block_table',
   'lift_blue_block_slider',
   'turn_off_lightbulb',
   'lift_red_block_table',
   'place_in_slider',
   'turn_off_lightbulb'],
  'emb': array([[[ 0.52925   ,  0.09874008,  0.15309368, ...,  0.6170549 ,
           -0.14766425,  0.21653481]],
  
         [[ 0.11232971, -0.14778604,  0.38895708, ...,  0.40465486,
            0.4906702 ,  0.2589114 ]],
  
         [[-0.04571903, -0.0061444 ,  0.27441305, ...,  0.07715372,
           -0.02765487,  0.1338684 ]],
  
         ...,
  
         [[-0.2222732 ,  0.02141502,  0.29

In [25]:
lang['language']['emb'].shape

(9, 1, 384)

In [26]:
len(lang['info']['indx'])

9

In [27]:
print(lang['language']['ann'][5])
print(lang['info']['indx'][5])

slide down the switch
(360571, 360635)


In [28]:
def display_frames(frames, title="", ms_per_frame=50):
    for i in range(len(frames)):
        if i == 1:
            _ = cv2.waitKey(0)
        cv2.imshow(title, frames[i])
        
        key = cv2.waitKey(50)#pauses for 3 seconds before fetching next image
        if key == 27:#if ESC is pressed, exit loop
            cv2.destroyAllWindows()
            break
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def collect_frames(start, stop):
    result = []
    for i in range(start, stop):
        
        result.append(np.load("episode_0" + str(i) + ".npz")["rgb_static"][:,:,::-1])
    return result


In [45]:
task_idx = 4
frames = collect_frames(lang['info']['indx'][task_idx][0], lang['info']['indx'][task_idx][1])
frames[0]
from PIL import Image
im = Image.fromarray(frames[-1][:,:,::-1])
im.save("../../../imgs/trajectory_figure_part_8.jpeg")
#display_frames(frames, "Static (200 x 200)")

In [16]:
lang['info']['indx'][task_idx][0]

360575

In [9]:
ep = np.load("episode_0360575.npz")

In [10]:
dict(ep)['scene_obs']

array([ 2.77312593e-01,  1.77498972e-01,  1.18748532e-19,  8.80000000e-02,
        1.00000000e+00,  1.00000000e+00,  6.21741856e-02, -1.05260515e-01,
        4.59990009e-01, -1.57079633e+00,  1.01304853e-09, -1.54320127e+00,
       -2.39174288e-01, -8.85086970e-02,  4.59965526e-01,  1.92448627e-05,
        3.39719465e-03, -1.62377881e+00,  1.62451444e-01, -1.15017202e-01,
        4.72010064e-01, -6.46464608e-02,  5.99302667e-02, -1.65834741e+00])

In [16]:
dict(ep)['depth_static']

array([[6.2016563, 6.195427 , 6.1895757, ..., 5.198949 , 5.1945705,
        5.1904564],
       [6.1921344, 6.1862893, 6.180091 , ..., 5.1922555, 5.188145 ,
        5.183785 ],
       [6.1826415, 6.1768146, 6.170635 , ..., 5.1855793, 5.1814795,
        5.1771307],
       ...,
       [4.195479 , 4.1979985, 4.2006893, ..., 4.147848 , 4.1452246,
        4.142441 ],
       [4.195479 , 4.1979985, 4.2006893, ..., 4.1435866, 4.1409683,
        4.1381903],
       [4.1958146, 4.1983347, 4.2006893, ..., 4.1393337, 4.136721 ,
        4.1339483]], dtype=float32)

In [17]:
print(np.max(dict(ep)['depth_static']))
print(np.min(dict(ep)['depth_static']))

6.2016563
3.695943


In [18]:
print(np.max(dict(ep)['depth_gripper']))
print(np.min(dict(ep)['depth_gripper']))

0.2771426
0.082857236


In [ ]:

[ 2.77312593e-01  1.77498972e-01  1.18748532e-19  8.80000000e-02
  1.00000000e+00  1.00000000e+00  6.21741856e-02 -1.05260515e-01
  4.59990009e-01 -1.57079633e+00  1.01304853e-09 -1.54320127e+00
 -2.39174288e-01 -8.85086970e-02  4.59965526e-01  1.92448627e-05
  3.39719465e-03 -1.62377881e+00  1.62451444e-01 -1.15017202e-01
  4.72010064e-01 -6.46464608e-02  5.99302667e-02 -1.65834741e+00]

In [20]:

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  print(cfg.env)

{'_target_': 'calvin_env.envs.play_table_env.PlayTableSimEnv', '_recursive_': False, 'cameras': '${cameras}', 'seed': 0, 'bullet_time_step': 240.0, 'use_vr': False, 'show_gui': False, 'robot_cfg': '${robot}', 'scene_cfg': '${scene}', 'use_scene_info': True, 'use_egl': False, 'control_freq': 30}


/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_data_collection.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/grail/miniconda3/envs/calvin_env/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'vr_input/vr_controller/vive': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


In [22]:
env = hydra.utils.instantiate(cfg.env)
observation = env.reset()
#The observation is given as a dictionary with different values
print(observation.keys())
for i in range(5):
  # The action consists in a pose displacement (position and orientation)
  action_displacement = np.random.uniform(low=-1, high=1, size=6)
  # And a binary gripper action, -1 for closing and 1 for oppening
  action_gripper = np.random.choice([-1, 1], size=1)
  action = np.concatenate((action_displacement, action_gripper), axis=-1)
  observation, reward, done, info = env.step(action)
  print(observation['scene_obs'])
  rgb = env.render(mode="rgb_array")[:,:,::-1]
  cv2_imshow(rgb)

disconnecting id 1 from server
dict_keys(['rgb_obs', 'depth_obs', 'robot_obs', 'scene_obs'])
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.44560290e-19
  0.00000000e+00  0.00000000e+00  9.89883188e-03 -1.35075945e-01
  4.59971892e-01  5.11323035e-05 -8.29153911e-04 -7.30876980e-01
 -1.63031718e-01  9.80176982e-02  4.60492255e-01 -3.80765860e-03
 -8.24435498e-04 -2.29269375e+00  7.98186152e-02  8.67648292e-02
  4.60475298e-01 -4.54503815e-03  8.67976304e-05  2.72751209e+00]


NameError: name 'cv2_imshow' is not defined

# Exploration of task_D_D dataset

In [12]:
%cd /home/grail/willaria_research/hobbes/dataset/task_D_D/validation

/home/grail/willaria_research/hobbes/dataset/task_D_D/validation


In [13]:
lang = np.load("lang_annotations/auto_lang_ann.npy", allow_pickle=True)
lang = dict(enumerate(lang.flatten(), 1))[1]
l = lang["language"]["ann"]
print(len(l))
print(l[0:5])

m = lang["language"]["task"]
print(len(m))
print(m[0:5])

1011
['turn on the green light', 'lift the pink block lying in the drawer', 'slide the door to the left', 'pick up the red block from the table', 'take the pink block and turn it right']
1011
['turn_on_led', 'lift_pink_block_drawer', 'move_slider_left', 'lift_red_block_table', 'rotate_pink_block_right']


In [14]:
# Count up number of each task
task_counts = {}
for task in lang['language']['task']:
    task_counts[task] = task_counts.get(task, 0) + 1
for task in task_counts:
    print(task_counts[task], '', task)

29  turn_on_led
30  lift_pink_block_drawer
32  move_slider_left
29  lift_red_block_table
30  rotate_pink_block_right
29  turn_off_lightbulb
33  turn_on_lightbulb
30  lift_blue_block_table
29  lift_blue_block_slider
29  close_drawer
29  lift_red_block_slider
32  turn_off_led
31  lift_red_block_drawer
30  push_red_block_left
30  place_in_slider
31  rotate_red_block_left
29  stack_block
31  lift_pink_block_slider
29  place_in_drawer
29  move_slider_right
30  open_drawer
29  unstack_block
29  push_red_block_right
30  push_into_drawer
29  push_blue_block_left
29  lift_pink_block_table
29  rotate_pink_block_left
30  lift_blue_block_drawer
29  push_pink_block_left
29  rotate_blue_block_left
29  rotate_blue_block_right
29  push_pink_block_right
30  rotate_red_block_right
29  push_blue_block_right


In [16]:
# Get all time sequences corresponding to a specific task
target_task = 'push_pink_block_right'
target_task_indxs = []
for (task, indx) in zip(lang['language']['task'], lang['info']['indx']):
    if task == target_task:
        target_task_indxs.append(indx)
print(target_task_indxs)

[(404279, 404343), (414511, 414575), (414530, 414573), (404283, 404347), (39644, 39708), (400519, 400557), (34360, 34404), (400514, 400560), (34360, 34408), (414525, 414571), (400499, 400563), (240731, 240777), (400527, 400562), (34350, 34414), (400504, 400568), (27327, 27372), (34371, 34405), (34363, 34410), (12837, 12881), (416833, 416879), (404291, 404355), (404282, 404346), (416833, 416881), (404281, 404345), (400501, 400565), (12835, 12878), (404284, 404348), (416808, 416872), (34372, 34407)]


In [44]:
# break here
a a aa a =a =a =a=a== 

SyntaxError: invalid syntax (1958689592.py, line 2)

In [13]:
lang["language"]["ann"]

['move the light switch to turn on the yellow light',
 'sweep the pink block to the right',
 'place the block in the sliding cabinet',
 'pick up the red block from the table',
 'in the slider grasp the blue block',
 'slide down the switch',
 'pick up the red block on the table',
 'place in slider',
 'turn off the light bulb']

In [14]:
lang["language"]["task"]

['turn_on_lightbulb',
 'push_pink_block_right',
 'place_in_slider',
 'lift_red_block_table',
 'lift_blue_block_slider',
 'turn_off_lightbulb',
 'lift_red_block_table',
 'place_in_slider',
 'turn_off_lightbulb']

In [15]:
lang["language"]["emb"].shape # 9 examples (corresponding to ann?), each is a 384 dim vector

(9, 1, 384)

In [16]:
episode = np.load("episode_0358482.npz")

In [17]:
for key in episode.keys():
    print(key + " : ", episode[key].shape)

actions :  (7,)
rel_actions :  (7,)
robot_obs :  (15,)
scene_obs :  (24,)
rgb_static :  (200, 200, 3)
rgb_gripper :  (84, 84, 3)
rgb_tactile :  (160, 120, 6)
depth_static :  (200, 200)
depth_gripper :  (84, 84)
depth_tactile :  (160, 120, 2)


In [18]:
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Gripper (84 x 84)", episode["rgb_gripper"]) #They have to have different names, or else override!
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Static (200 x 200)", episode["depth_static"]) 
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Gripper (84 x 84)", episode["depth_gripper"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread (0x2fcb6b0).
Cannot move to target thread (0x7c00560)

QObject::moveToThread: Current thread (0x7c00560) is not the object's thread

In [15]:
episode = np.load("episode_0358482.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Gripper (84 x 84)", episode["rgb_gripper"]) #They have to have different names, or else override!
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Static (200 x 200)", episode["depth_static"]) 
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Depth Gripper (84 x 84)", episode["depth_gripper"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread

In [17]:
episode = np.load("episode_0358482.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358483.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358484.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()
episode = np.load("episode_0358485.npz")
cv2.imshow("Static (200 x 200)", episode["rgb_static"])
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread (0x2254870).
Cannot move to target thread (0x74083e0)

QObject::moveToThread: Current thread (0x74083e0) is not the object's thread